# Preprocessing
This notebook contains all code for the preprocessing of the KDD Cup 98 datasets.
* Splits into learning and test
* Learns the transformation pipeline on the learning dataset for future use
* Prepares the data for model fitting

This will be done with scikit-learn's transforming framework in order to ensure all transformations are applied identically on training, test and validation datasets.


The transformations are 'learned' on the training dataset and then applied to the test dataset and new data later on.

In [1]:
%load_ext autoreload

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

os.chdir("../") # Change to project root
import util.data_loader as dl
from kdd98.transformers import *

In [3]:
# plto config
%matplotlib inline
sns.set(color_codes=True)
sns.set_style('ticks')
plt.rcParams['figure.figsize'] = [40, 40]

## Loading the learning dataset


Set working directory to main code folder

In [4]:
%autoreload 2 # automatically reloads modules
data_loader = dl.KDD98DataLoader("cup98LRN.txt")
learning_raw = data_loader.get_dataset()

## Splitting into training- and test dataset

Before applying *any* transformations, the dataset will be split 80/20 into a learning and test set.

Let's look at feature TARGET_B, which describes whether a person has donated or not:

In [5]:
learning_raw.TARGET_B.value_counts(normalize=True) # 5 % of recipients have donated.

0    0.949241
1    0.050759
Name: TARGET_B, dtype: float64

We want to preserve this ratio in the split datasets. scikit-learn provides a class for achieving this.

In [6]:
from config import App
seed = App.config("random_seed")
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size=0.8, random_state=seed)
for learn_index, test_index in splitter.split(learning_raw, learning_raw.TARGET_B.astype('int')):
    l_i = learn_index
    t_i = test_index
    kdd_learn = learning_raw.iloc[learn_index]
    kdd_test = learning_raw.iloc[test_index]
    

Now, check that the two sets are really disjoint

In [7]:
set(kdd_learn.index).intersection(kdd_test.index)

set()

Check the frequencies of the donors in the sets:

In [8]:
kdd_learn['TARGET_B'].value_counts()/len(kdd_learn.index)

0    0.949246
1    0.050754
Name: TARGET_B, dtype: float64

In [9]:
kdd_test['TARGET_B'].value_counts()/len(kdd_test.index)

0    0.949222
1    0.050778
Name: TARGET_B, dtype: float64

## Applying the cleaning pipeline

In [10]:
cleaning_pipeline = ColumnTransformer([
    ("bool_x_bl",
     BooleanFeatureRecode(value_map={'true': 'X', 'false': ' '}),
     ['PEPSTRFL', 'NOEXCH', 'MAJOR', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP']
     ),
    ("bool_y_n",
     BooleanFeatureRecode(value_map={'true': 'Y', 'false': 'N'}),
     ['COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO',
      'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN',  'BOATS', 'WALKER', 'KIDSTUFF',
      'CARDS', 'PLATES']
     ),
    ("bool_e_i",
     BooleanFeatureRecode(value_map={'true': "E", 'false': 'I'}),
     ['AGEFLAG']
     ),
    ("bool_h_u",
     BooleanFeatureRecode(value_map={'true': "H", 'false': 'U'}),
     ['HOMEOWNR']),
    ("bool_b_bl",
     BooleanFeatureRecode(value_map={'true': 'B', 'false': ' '}),
     ['MAILCODE']
     ),
    ("bool_1_0",
     BooleanFeatureRecode(value_map={'true': '1', 'false': '0'}),
     ['HPHONE_D', 'TARGET_B']
     ),
    ("truncate_zip",
     ZipCodeFormatter(),
     ['ZIP']
     ),
    ("hash_osource",
     HashingEncoder(),
     dl.nominal_features[0]
    ),
    ("rfa_spread",
     MultiByteExtract(["R", "F", "A"]),
     dl.nominal_features[1:]),
    ("drop_old_rfa",
     "drop",
     dl.nominal_features),
    ("months_to_donation",
     Pipeline(steps=[
         ("parse_donation_dates", ParseDates(treat_errors='ignore')),
         ("months_for_donation", MonthsToDonation())
         ]),
     dl.don_hist_dates
     ),
    ("drop_adate_rdate",
     "drop",
     dl.don_hist_dates
    ),
    ("relative_years",
     Pipeline(steps=[
         ("parse_remaining_dates", ParseDates(treat_errors='ignore')),
         ("rel_years", ComputeAge())
     ]),
    ['ODATEDW', 'DOB'])
])

## Separating features and label

First, we separate the features from the labels. We will also remove the label "TARGET_B", which is an indicator variable for donors that is no longer of interest

**All preprocessing is performed on *kdd_learn_feat***

In [ ]:
kdd_learn_feat = kdd_learn.drop(['TARGET_B', 'TARGET_D'],axis=1)
kdd_learn_label = kdd_learn['TARGET_D'].copy()

## Exploring strategies for specific feature types

* Noisy data: Correction of data entry / formatting errors
    - These errors must be corrected without excluding the records in question
* Missing data: Has to be inferred from known values
    - (e.g., mean, median, mode, a modeled value).
    - One exception to this rule is the attributes containing 99.5 percent or more missings. These are to be dropped
* Sparse data: Events actually represented in given data make only a very small subset of the event space are to be dropped
* Constant values are to be dropped

### Constant and Sparse Features

Features where only one value is present and those where the majority is empty are to be dropped.


In [ ]:
const_sparse_transformer = DropSparseLowVar(keep_anyways=["RAMNT_\d{1,2}", "MONTHS_TO_DONATION_\d{1,2}"])
cs = const_sparse_transformer.fit(learning)
cs = const_sparse_transformer.fit_transform(learning)
set(cs.columns)
const_sparse_transformer.get_feature_names()

### Numerical features

In [ ]:
numerical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

### Categorical features

### Remaining object features

In [ ]:
objects = learning_raw.select_dtypes(include='object').columns
print(objects)

In [ ]:
for f in objects:
    print(f+": "+learning_raw[f].unique())

These are two types:

* ZIP: Malformed zip codes. Some have a dash at the end, which has to be removed.
* Multibyte values. These can be extracted into separate features bytewise. However, this is done in feature extraction later on

## Preprocessing Pipeline

It is now time to construct the preprocessing pipeline. A set of transforming operations is concatenated to a sequence of operations. This pipeline is the learned on the learning dataset. All transformations to the learning dataset will then later be applied to the test dataset and to new data.

In [ ]:
numerical_feats = list(kdd_learn_feat.select_dtypes(include=np.number).columns)
categorical_feats = list(kdd_learn_feat.select_dtypes(include=np.number).columns)

With all categories now properly formatted, it is time for one-hot encoding. The sklearn pipeline also has an impute transformation. NaN's get their own level, "missing". This step results in a huge increase in the dimension of the feature space. It is also heavy on computation.

In [ ]:
cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("one_hot",  OneHotEncoder(impute_missing=True,use_cat_names=True,return_df=True))
])

categories_transformer = ColumnTransformer([
    ("cat_encoder",
     cat_pipe,
     list(kdd_learn_feat.select_dtypes(include="category").columns))
])

Interests and donations

In [ ]:
data = learning_raw.loc[:,dl.interest_features+["TARGET_D"]].fillna(0)
interests = pd.melt(data,value_vars=dl.interest_features, value_name="Interest")
data.head()

Features with constant values:

## Feature Selection
Meant to reduce dimensionality by selecting only features that are 'interesting enough' to be considered in order to boost performance of calculations / improve accuracy of the estimator
- By variance threshold
- Recursive Feature Elimination by Cross-Validation
- L1-based feature selection (Logistic Regression, Lasso, SVM)
- Tree-based feature selection

See [scikit-learn: feature selection](http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection)

### Removing constant features (zero variance)

sklearn.feature_selection_variance_threshold

In [ ]:
for column in learning.columns:
        if len(learning[column].unique()) == 1:
            print(column)

### Sparse Features

In [ ]:
sparse_features = []
for column in learning:
    top_freq = learning[column].value_counts(normalize=True).iloc[0]
    if top_freq > 0.995:
        sparse_features.append(column)
        print(column+" has a top frequency of: " + str(top_freq))
        print(learning[column].value_counts(normalize=True))

In [ ]:
sparse_features

### Advanced approaches

* If overfitting is a problem, ensemble-learning or tree learning can be used to find important features, then apply SelectFromModel before the actual estimator. See http://scikit-learn.org/stable/modules/feature_selection.html

## Feature Extraction
All explanatory fields have to be numerical for the subsequent operations with scikit-learn. Here, the necessary feature extractions are performed.

See [scikit-learn: feature extraction](http://scikit-learn.org/stable/modules/feature_extraction.html)

In [ ]:
import pandas as pd

In [ ]:
symbolic_features = []
symbolic_features.append(tds.SymbolicFeatureSpreader(
    "DOMAIN", ["U", "S"])) #Urbanicity, SocioEconomicStatus
# RFA_2 is already spread out
for i in range(3, 25):
    feature = "_".join(["RFA", str(i)])
    symbolic_features.append(tds.SymbolicFeatureSpreader(
        feature, ["R", "F", "A"])) # Recency, Frequency, Amount

spread_multibyte = pd.DataFrame(index=learning_raw.index)
for f in symbolic_features:
    f.set_tidy_dataset_ref(learning_raw)
    spread_multibyte = pd.concat([spread_multibyte,f.spread(inplace=False)],axis=1)

In [ ]:
spread_multibyte.info()

# PCA

A first look at important features

In [ ]:
from sklearn import decomposition

In [ ]:
X = learning.drop(["TARGET_B","TARGET_D"],axis=1)

In [ ]:
n_comp = 3
pca = decomposition.PCA(n_components = n_comp)
pca.fit(X)
result = pd.DataFrame(pca.transform(X), columns=["PCA%i" % i for i in range(n_comp)], index=X.index)

In [ ]:
import cProfile
domain_spreader = tds.SymbolicFieldToDummies(learning,"RFA_24",["Recency", "Frequency", "Amount"])
cProfile.run('domain_spreader.spread()', sort='time')

In [ ]:
learning.head()

In [ ]:
import os
import numpy as np
import sys
os.getcwd()
proj_dir = os.path.split(os.getcwd())[0]
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

In [ ]:
import eda.tidy_dataset as tds
tidy = tds.TidyDataset("cup98LRN.txt")

In [ ]:
raw = tidy.get_raw_data()

In [ ]:
spreader = tds.SymbolicFieldToDummies(
    raw, "RFA_24", ["Recency", "Frequency", "Amount"])
spreader.spread()